In [47]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LinearRegression
# from sklearn.svm import SVR

# from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error
import random
random.seed( 10 )

In [48]:
path = "D:\\Web\\game\\"      # set path

train9 = pd.read_csv(path + 'train9.csv')
train1 = pd.read_csv(path + 'train1.csv')

test9 = pd.read_csv(path + 'test9.csv')
test1 = pd.read_csv(path + 'test1.csv')

hero = pd.read_csv(path + 'hero_data.csv')

In [49]:
train9['win_ratio'] = train9.num_wins/train9.num_games
#train1['win_ratio'] = train1.num_wins/train1.num_games
test9['win_ratio'] = test9.num_wins/test9.num_games
#test1['win_ratio'] = test1.num_wins/test1.num_games

In [50]:
tf = CountVectorizer()
y = tf.fit_transform(hero.roles.str.replace(':', ','))

roles = pd.DataFrame(y.toarray(),columns=tf.vocabulary_)

hero_1=pd.concat([hero, roles], axis =1)

In [51]:
hero_1.drop(['roles','base_health','base_mana'],axis=1 , inplace=True)
hero_1 = pd.get_dummies(hero_1)

In [52]:
train_9 = train9.merge(hero_1)
train_1 = train1.merge(hero_1)
test_9 = test9.merge(hero_1)
test_1 = test1.merge(hero_1)

In [53]:
# train kda mean and variance

user_kda = {}
user_var = {}
user_min = {}
user_max = {}

hero_kda = {}
hero_var = {}

a = train_9.user_id.unique()
for i in a:
    user_kda[i] = train_9[train_9.user_id==i].kda_ratio.mean()
for i in a:
    user_var[i] = train_9[train_9.user_id==i].kda_ratio.var()
for i  in a:
    user_min[i] = train_9[train_9.user_id ==i ].kda_ratio.min()
for i in a:
    user_max[i] = train_9[train_9.user_id == i].kda_ratio.max()
    
b = train_9.hero_id.unique()
for i in b:
    hero_kda[i] = train_9[train_9.hero_id==i].kda_ratio.mean()
for i in b:
    hero_var[i] = train_9[train_9.hero_id==i].kda_ratio.var()

train_9['user_kda'] = [user_kda[i] for i in train_9.user_id]
#train_9['user_var'] = [user_var[i] for i in train_9.user_id]

train_9['hero_kda'] = [hero_kda[i] for i in train_9.hero_id]
#train_9['hero_var'] = [hero_var[i] for i in train_9.hero_id]


train_1['user_kda'] = [user_kda[i] for i in train_1.user_id]
#train_1['user_var'] = [user_var[i] for i in train_1.user_id]

train_1['hero_kda'] = [hero_kda[i] for i in train_1.hero_id]
#train_1['hero_var'] = [hero_var[i] for i in train_1.hero_id]

# train win ratio

win_ratio = {}
win_var = {}

a = train_9.user_id.unique()
for i in a:
    win_ratio[i] = train_9[train_9.user_id==i].win_ratio.mean()
for i in a:
    win_var[i] = train_9[train_9.user_id==i].win_ratio.var()
    

train_9['win_ratio1'] = [win_ratio[i] for i in train_9.user_id]
train_9['win_var'] = [win_var[i] for i in train_9.user_id]

train_1['win_ratio1'] = [win_ratio[i] for i in train_1.user_id]
train_1['win_var'] = [win_var[i] for i in train_1.user_id]

In [54]:
# test kda mean var

user_kda = {}
user_var = {}
user_min = {}
user_max = {}

hero_kda = {}
hero_var = {}

a = test_9.user_id.unique()
for i in a:
    user_kda[i] = test_9[test_9.user_id==i].kda_ratio.mean()
for i in a:
    user_var[i] = test_9[test_9.user_id==i].kda_ratio.var()
for i  in a:
    user_min[i] = test_9[test_9.user_id ==i ].kda_ratio.min()
for i in a:
    user_max[i] = test_9[test_9.user_id == i].kda_ratio.max()
b = test_9.hero_id.unique()
for i in b:
    hero_kda[i] = test_9[test_9.hero_id==i].kda_ratio.mean()
for i in b:
    hero_var[i] = test_9[test_9.hero_id==i].kda_ratio.var()

test_9['user_kda'] = [user_kda[i] for i in test_9.user_id]
#test_9['user_var'] = [user_var[i] for i in test_9.user_id]

test_9['hero_kda'] = [hero_kda[i] for i in test_9.hero_id]
#test_9['hero_var'] = [hero_var[i] for i in test_9.hero_id]

test_1['user_kda'] = [user_kda[i] for i in test_1.user_id]
#test_1['user_var'] = [user_var[i] for i in test_1.user_id]

test_1['hero_kda'] = [hero_kda[i] for i in test_1.hero_id]
#test_1['hero_var'] = [hero_var[i] for i in test_1.hero_id]

win_ratio = {}
win_var = {}

a = test_9.user_id.unique()
for i in a:
    win_ratio[i] = test_9[test_9.user_id==i].win_ratio.mean()
for i in a:
    win_var[i] = test_9[test_9.user_id==i].win_ratio.var()
    

test_9['win_ratio1'] = [win_ratio[i] for i in test_9.user_id]
test_9['win_var'] = [win_var[i] for i in test_9.user_id]

test_1['win_ratio1'] = [win_ratio[i] for i in test_1.user_id]
test_1['win_var'] = [win_var[i] for i in test_1.user_id]


In [55]:
print(train_9.shape)
train_9 = pd.concat([train_9, test_9], axis = 0)
X_train = train_9.drop(['id','kda_ratio','hero_id','user_id','win_ratio','num_wins'],axis=1)
y_train = train_9.kda_ratio
X_test = train_1.drop(['id','kda_ratio','hero_id','user_id','num_wins'],axis=1)
y_test = train_1.kda_ratio
X_test2 = test_1.drop(['id','hero_id','user_id'],axis=1)
print(train_9.shape)

(18846, 42)
(26928, 42)


In [56]:
# #step = [ 'num_games' , 'base_health_regen' , 'base_armor' , 
#     'base_attack_max' , 'base_intelligence' , 'strength_gain' , 'intelligence_gain' , 
#     'attack_range' , 'projectile_speed' , 'attack_rate' , 'turn_rate' , 
#     'escape' , 'durable' , 'disabler' , 'nuker' , 'initiator' , 'pusher' , 
#     'support' , 'carry' , 'primary_attr_agi' , 'primary_attr_int' , 'attack_type_Melee' , 
#     'user_kda'  , 'hero_kda'  , 'win_ratio1' , 
#     'win_var'  ]

X_train_S = X_train
X_test_S = X_test
X_test2_S = X_test2

sc = StandardScaler()
X_train_S = sc.fit_transform(X_train_S)
X_test_S = sc.transform(X_test_S)
X_test2_S = sc.transform(X_test2_S)

model = LinearRegression()
model.fit(X_train_S, y_train) 

train_pred = model.predict(X_test_S)
test_pred1  = model.predict(X_test2_S)
train_rmse = np.sqrt(mean_squared_error(train_pred,y_test))
print(train_rmse)

593.354895945


In [57]:
#columns = X_train.columns
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_test2 = sc.transform(X_test2)

from sklearn.neighbors import KNeighborsRegressor

model2 = KNeighborsRegressor(n_neighbors=9)
model2.fit(X_train, y_train) 
train_pred2 = model2.predict(X_test)
test_pred2  = model2.predict(X_test2)
train_rmse = np.sqrt(mean_squared_error(train_pred2,y_test))
print(train_rmse)

593.974578299


In [60]:
print(np.sqrt(mean_squared_error(y_test,  (train_pred + train_pred2)/2)))

577.735559196


In [61]:
kda_ratio = (test_pred1*3 + test_pred2*2 )/5

d = pd.DataFrame({ 'id':test_1.id,
                  'kda_ratio':kda_ratio
                 })
d.to_csv(path+'linearknn.csv',index = None)